# Intégrer une interface utilisateur 

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QLineEdit, QPushButton, QGraphicsView, QGraphicsScene, QGraphicsRectItem
from PyQt5.QtGui import QBrush, QPen, QColor , QPainter
from PyQt5.QtCore import Qt
from fonctions import *

In [2]:
class RobotTrainingApp(QMainWindow):
    def __init__(self):
        super().__init__()

        # Window settings
        self.setWindowTitle("Q-Learning Dashboard")
        self.setGeometry(100, 100, 1200, 600)

        # Initialize UI elements
        self.init_ui()

    def init_ui(self):
        # Main layout
        main_layout = QHBoxLayout()

        # Left panel for user inputs
        self.left_panel = self.create_left_panel()
        main_layout.addLayout(self.left_panel, 1)

        # Right panel for environment visualization
        self.right_panel = self.create_right_panel()
        main_layout.addWidget(self.graphics_view, 2)

        # Set central widget
        central_widget = QWidget()
        central_widget.setLayout(main_layout)
        self.setCentralWidget(central_widget)

    def create_left_panel(self):
        layout = QVBoxLayout()

        # Input fields for environment size, source, and objective
        layout.addWidget(QLabel("Environment Size:"))
        self.env_size_input = QLineEdit("10")
        layout.addWidget(self.env_size_input)

        layout.addWidget(QLabel("Source (x, y):"))
        self.source_input = QLineEdit("0,0")
        layout.addWidget(self.source_input)

        layout.addWidget(QLabel("Objective (x, y):"))
        self.objective_input = QLineEdit("9,9")
        layout.addWidget(self.objective_input)

        layout.addWidget(QLabel("Obstacles (e.g., 1,1;2,2):"))
        self.obstacles_input = QLineEdit("(0,5;0,6;3,3;3,4;6,5;6,6)")
        layout.addWidget(self.obstacles_input)

        layout.addWidget(QLabel("Strawberries (e.g., 4,4;5,5):"))
        self.strawberries_input = QLineEdit("(2,5;8,7)")
        layout.addWidget(self.strawberries_input)

        layout.addWidget(QLabel("Alpha:"))
        self.alpha_input = QLineEdit("0.8")
        layout.addWidget(self.alpha_input)

        layout.addWidget(QLabel("Epsilon:"))
        self.epsilon_input = QLineEdit("0.3")
        layout.addWidget(self.epsilon_input)

        layout.addWidget(QLabel("Gamma:"))
        self.gamma_input = QLineEdit("0.9")
        layout.addWidget(self.gamma_input)

        layout.addWidget(QLabel("Episodes:"))
        self.episodes_input = QLineEdit("1000")
        layout.addWidget(self.episodes_input)

        # Buttons
        self.create_env_button = QPushButton("Create Environment")
        self.create_env_button.clicked.connect(self.create_environment)
        layout.addWidget(self.create_env_button)

        self.start_training_button = QPushButton("Start Training")
        self.start_training_button.clicked.connect(self.start_training)
        layout.addWidget(self.start_training_button)

        return layout

    def create_right_panel(self):
        # Graphics view for environment visualization
        self.scene = QGraphicsScene()
        self.graphics_view = QGraphicsView(self.scene)
        self.graphics_view.setRenderHint(QPainter.Antialiasing, True)
        return self.graphics_view

    def create_environment(self):
        # Read environment size
        grid_size = int(self.env_size_input.text())
        self.grid_size = grid_size
        self.cell_size = 500 // grid_size

        # Clear the scene and draw grid
        self.scene.clear()

        for x in range(grid_size):
            for y in range(grid_size):
                rect = QGraphicsRectItem(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
                rect.setBrush(Qt.white)
                rect.setPen(QPen(Qt.gray))
                self.scene.addItem(rect)

        # Read user inputs for obstacles and strawberries
        self.source = tuple(map(int, self.source_input.text().split(',')))
        self.objective = tuple(map(int, self.objective_input.text().split(',')))

        # Obstacles
        self.obstacles = []
        if self.obstacles_input.text():
            # Remove parentheses and whitespace, then split the coordinates
            obstacles_text = self.obstacles_input.text().replace("(", "").replace(")", "").replace(" ", "")
            self.obstacles = [tuple(map(int, coord.split(','))) for coord in obstacles_text.split(';')]

        for obs in self.obstacles:
            rect = QGraphicsRectItem(obs[1] * self.cell_size, obs[0] * self.cell_size, self.cell_size, self.cell_size)
            rect.setBrush(Qt.black)
            self.scene.addItem(rect)

        # Strawberries
        self.strawberries = []
        if self.strawberries_input.text():
            # Remove parentheses and whitespace, then split the coordinates
            strawberries_text = self.strawberries_input.text().replace("(", "").replace(")", "").replace(" ", "")
            self.strawberries = [tuple(map(int, coord.split(','))) for coord in strawberries_text.split(';')]

        for straw in self.strawberries:
            rect = QGraphicsRectItem(straw[1] * self.cell_size, straw[0] * self.cell_size, self.cell_size, self.cell_size)
            rect.setBrush(Qt.red)
            self.scene.addItem(rect)

        # Mark source and objective
        rect = QGraphicsRectItem(self.source[1] * self.cell_size, self.source[0] * self.cell_size, self.cell_size, self.cell_size)
        rect.setBrush(Qt.green)
        self.scene.addItem(rect)

        rect = QGraphicsRectItem(self.objective[1] * self.cell_size, self.objective[0] * self.cell_size, self.cell_size, self.cell_size)
        rect.setBrush(Qt.blue)
        self.scene.addItem(rect)


    def start_training(self):
        # Retrieve training parameters
        episodes = int(self.episodes_input.text())
        alpha = float(self.alpha_input.text())
        epsilon = float(self.epsilon_input.text())
        gamma = float(self.gamma_input.text())

        # Initialize Q-table
        q_table = np.zeros((self.grid_size, self.grid_size, 4))

        # Perform Q-learning
        q_table = q_learning(episodes, alpha, gamma, epsilon, q_table, strawberries=self.strawberries)

        # Find optimal path
        optimal_path = find_optimal_path(q_table, self.source, self.objective)

        # Visualize the optimal path
        self.visualize_optimal_path(optimal_path)

    def visualize_optimal_path(self, path):
        for i in range(len(path) - 1):
            start = path[i]
            end = path[i + 1]
            line = self.scene.addLine(
                start[1] * self.cell_size + self.cell_size // 2,
                start[0] * self.cell_size + self.cell_size // 2,
                end[1] * self.cell_size + self.cell_size // 2,
                end[0] * self.cell_size + self.cell_size // 2,
                QPen(QColor(Qt.blue))
            )
            self.scene.addItem(line)





In [3]:
app = QApplication(sys.argv)
window = RobotTrainingApp()
window.show()
sys.exit(app.exec())

SystemExit: 0

c:\ProgramData\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
